In [1]:
import os
import numpy as np
import pandas as pd
from astropy.table import Table

from lvmdrp import path, __version__ as drpver
from lvmdrp.utils import metadata as md
from lvmdrp.functions import run_drp as drp

from lvmdrp.functions import imageMethod

ORIG_MASTER_DIR = os.getenv("LVM_MASTER_DIR")
MASTER_CON_LAMPS = {"b": "ldls", "r": "ldls", "z": "quartz"}
MASTER_ARC_LAMPS = {"b": "hgne", "r": "neon", "z": "neon"}
SLITMAP = Table(drp.fibermap.data)

In [2]:
frames_table = md.get_metadata(tileid="*", mjd=60177)
frames_table.query("imagetyp == 'bias' | imagetyp == 'dark' | imagetyp == 'flat' | imagetyp == 'arc'", inplace=True)
frames_table = frames_table.loc[frames_table.name.str.contains("sdR")]

[INFO]: loading metadata store at /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/raw_metadata.hdf5
[INFO]: found 558 frames in store '/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/raw_metadata.hdf5'
[INFO]: number of frames after filtering 558
[INFO]: total number of frames found 558


In [3]:
masters_mjd = frames_table.mjd.min()
masters_path = os.path.join(ORIG_MASTER_DIR, f"{masters_mjd}")
os.makedirs(masters_path, exist_ok=True)

In [4]:
bias_analogs = frames_table.query("imagetyp=='bias'").groupby(["mjd", "camera"])

mbias_paths = []
for mjd, camera in bias_analogs.groups:
    biases = bias_analogs.get_group((mjd, camera))
    print(biases.to_string())
    
    # define master frame path
    mbias_path = os.path.join(masters_path, f"lvm-mbias-{camera}.fits")
    mbias_paths.append(mbias_path)
    
    # preprocess biases
    pbias_paths = []
    for bias in biases.to_dict("records"):
        bias_path = path.full("lvm_raw", camspec=bias["camera"], **bias)
        pbias_path = path.full("lvm_anc", drpver=drpver, kind="p", imagetype=bias["imagetyp"], **bias)
        pbias_paths.append(pbias_path)
        os.makedirs(os.path.dirname(pbias_path), exist_ok=True)
        
        imageMethod.preproc_raw_frame(in_image=bias_path, out_image=pbias_path)

    # create master bias
    imageMethod.create_master_frame(
        in_images=pbias_paths,
        out_image=mbias_path,
        master_mjd=masters_mjd
    )

# cache new masters metadata into HDF5 files
new_masters = md.extract_metadata(frames_paths=mbias_paths)
md.add_masters(new_masters)

    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
24     s    1111  60177  60177     bias  sp1     b1    3447      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00003447.fits
57     s    1111  60177  60177     bias  sp1     b1    3440      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00003440.fits
107    s    1111  60177  60177     bias  sp1     b1    3448      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00003448.fits
178    s    1111  60177  60177     bias  sp1     b1    3444      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00003444.fits
184    s    1111  60177  60177     bias  sp1     b1    3442      0.0  

[INFO]: starting preprocessing of raw image 'sdR-s-b1-00003447.fits.gz'
[INFO]: using header IMAGETYP = 'bias'
[INFO]: exposure time 0.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.71, 2.71, 2.68, 2.69] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 974.00 +/- 1.10 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 981.00 +/- 1.13 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 982.00 +/- 1.27 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 982.00 +/- 1.28 (ADU)
[INFO]: using header RDNOISE = [3.0, 3.0, 2.8, 2.8] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: building pixel mask
[INFO]: 1348 (0.0081 %) pixels masked
[INFO]: replacing 1348 masked 

    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
66     s    1111  60177  60177     bias  sp2     b2    3442      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00003442.fits
102    s    1111  60177  60177     bias  sp2     b2    3448      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00003448.fits
240    s    1111  60177  60177     bias  sp2     b2    3446      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00003446.fits
270    s    1111  60177  60177     bias  sp2     b2    3443      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00003443.fits
314    s    1111  60177  60177     bias  sp2     b2    3441      0.0  

[INFO]: using header IMAGETYP = 'bias'
[INFO]: exposure time 0.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.45, 2.46, 2.46, 2.5] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 1003.00 +/- 1.13 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 999.00 +/- 1.13 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 1007.00 +/- 1.25 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 1002.00 +/- 1.28 (ADU)
[INFO]: using header RDNOISE = [3.0, 3.0, 2.8, 2.8] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: building pixel mask
[INFO]: 5120 (0.031 %) pixels masked
[INFO]: replacing 5120 masked pixels with NaNs
[INFO]: writing preprocessed image to lvm-pbias-b2-000

    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
105    s    1111  60177  60177     bias  sp3     b3    3447      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b3-00003447.fits
156    s    1111  60177  60177     bias  sp3     b3    3445      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b3-00003445.fits
218    s    1111  60177  60177     bias  sp3     b3    3441      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b3-00003441.fits
237    s    1111  60177  60177     bias  sp3     b3    3442      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b3-00003442.fits
317    s    1111  60177  60177     bias  sp3     b3    3446      0.0  

...

[INFO]: starting preprocessing of raw image 'sdR-s-z1-00003442.fits.gz'
[INFO]: using header IMAGETYP = 'bias'
[INFO]: exposure time 0.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.76, 2.65, 2.89, 2.78] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 980.00 +/- 1.07 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 979.00 +/- 1.08 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 989.00 +/- 1.13 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 990.00 +/- 1.16 (ADU)
[INFO]: using header RDNOISE = [3.6, 3.6, 3.4, 3.6] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: building pixel mask
[INFO]: 0 (0 %) pixels masked
[INFO]: replacing 0 masked pixels with

    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
42     s    1111  60177  60177     bias  sp2     z2    3441      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z2-00003441.fits
51     s    1111  60177  60177     bias  sp2     z2    3446      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z2-00003446.fits
64     s    1111  60177  60177     bias  sp2     z2    3445      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z2-00003445.fits
150    s    1111  60177  60177     bias  sp2     z2    3444      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z2-00003444.fits
180    s    1111  60177  60177     bias  sp2     z2    3440      0.0  

[INFO]: using header IMAGETYP = 'bias'
[INFO]: exposure time 0.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.6, 2.65, 2.57, 2.72] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 988.00 +/- 1.14 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 982.00 +/- 1.19 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 998.00 +/- 1.16 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 998.00 +/- 1.20 (ADU)
[INFO]: using header RDNOISE = [3.6, 3.6, 3.4, 3.6] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: building pixel mask
[INFO]: 2454 (0.015 %) pixels masked
[INFO]: replacing 2454 masked pixels with NaNs
[INFO]: writing preprocessed image to lvm-pbias-z2-000034

    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
37     s    1111  60177  60177     bias  sp3     z3    3442      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z3-00003442.fits
87     s    1111  60177  60177     bias  sp3     z3    3448      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z3-00003448.fits
177    s    1111  60177  60177     bias  sp3     z3    3441      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z3-00003441.fits
202    s    1111  60177  60177     bias  sp3     z3    3445      0.0  False  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z3-00003445.fits
210    s    1111  60177  60177     bias  sp3     z3    3440      0.0  

[INFO]: using header IMAGETYP = 'bias'
[INFO]: exposure time 0.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.6, 2.65, 2.57, 2.72] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 1009.00 +/- 1.16 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 1012.00 +/- 1.14 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 1012.00 +/- 1.26 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 1010.00 +/- 1.28 (ADU)
[INFO]: using header RDNOISE = [3.6, 3.6, 3.4, 3.6] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: building pixel mask
[INFO]: 0 (0 %) pixels masked
[INFO]: replacing 0 masked pixels with NaNs
[INFO]: writing preprocessed image to lvm-pbias-z3-00003442.fit

In [5]:
dark_analogs = frames_table.query("imagetyp=='dark'").groupby(["mjd", "camera"])

mdark_paths = []
for mjd, camera in dark_analogs.groups:
    darks = dark_analogs.get_group((mjd, camera))
    print(darks.to_string())
    
    # define master paths for target frames
    mbias_path = os.path.join(masters_path, f"lvm-mbias-{camera}.fits")
    
    # define master frame path
    mdark_path = os.path.join(masters_path, f"lvm-mdark-{camera}.fits")
    mdark_paths.append(mdark_path)
    
    # preprocess and detrend darks
    ddark_paths = []
    for dark in darks.to_dict("records"):
        dark_path = path.full("lvm_raw", camspec=dark["camera"], **dark)
        pdark_path = path.full("lvm_anc", drpver=drpver, kind="p", imagetype=dark["imagetyp"], **dark)
        ddark_path = path.full("lvm_anc", drpver=drpver, kind="d", imagetype=dark["imagetyp"], **dark)
        ddark_paths.append(ddark_path)
        
        imageMethod.preproc_raw_frame(in_image=dark_path, out_image=pdark_path)
        imageMethod.detrend_frame(in_image=pdark_path, out_image=ddark_path, in_bias=mbias_path)

    # create master dark
    imageMethod.create_master_frame(in_images=ddark_paths, out_image=mdark_path, master_mjd=masters_mjd)

# cache new masters metadata into HDF5 files
new_masters = md.extract_metadata(frames_paths=mdark_paths)
md.add_masters(new_masters)

[WARNING]: zero paths given, nothing to do
[INFO]: loading metadata store at /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/master_metadata.hdf5
[INFO]: updating metadata store for masters with 0 new rows
[INFO]: final number of rows 48
[INFO]: writing master metadata store to disk


In [4]:
flat_analogs = frames_table.query("imagetyp=='flat'").groupby(["camera"])

mflat_paths = []
for camera in flat_analogs.groups:
    flats = flat_analogs.get_group(camera)
    flat = flats.iloc[0].to_dict()
    
    lamp = MASTER_CON_LAMPS[camera[0]]
    flats = flats.query(lamp)

    print(flats.to_string())
    
    # define master paths for target frames
    mpixmask_path = os.path.join(masters_path, f"lvm-mpixmask-{camera}.fits")
    mbias_path = os.path.join(masters_path, f"lvm-mbias-{camera}.fits")
    mdark_path = os.path.join(masters_path, f"lvm-mdark-{camera}.fits")
    
    # define master frame path
    mflat_path = path.full("lvm_master", drpver=drpver, tileid=flat["tileid"], mjd=masters_mjd, kind=f"mflat_{lamp}", camera=camera)
    mflat_paths.append(mflat_path)
    os.makedirs(os.path.dirname(mflat_path), exist_ok=True)
    
    # preprocess and detrend flats
    dflat_paths = []
    for flat in flats.to_dict("records"):
        flat_path = path.full("lvm_raw", camspec=flat["camera"], **flat)
        pflat_path = path.full("lvm_anc", drpver=drpver, kind="p", imagetype=flat["imagetyp"], **flat)
        dflat_path = path.full("lvm_anc", drpver=drpver, kind="d", imagetype=flat["imagetyp"], **flat)
        dflat_paths.append(dflat_path)
        
        imageMethod.preproc_raw_frame(in_image=flat_path, out_image=pflat_path, in_mask=mpixmask_path)
        imageMethod.detrend_frame(in_image=pflat_path, out_image=dflat_path, in_bias=mbias_path, in_dark=mdark_path, in_slitmap=SLITMAP)

    # create master flat (2D image)
    imageMethod.create_master_frame(in_images=dflat_paths, out_image=mflat_path, master_mjd=masters_mjd)

# cache new masters metadata into HDF5 files
new_masters = md.extract_metadata(frames_paths=mflat_paths)
md.add_masters(new_masters)

[INFO]: starting preprocessing of raw image 'sdR-s-b1-00003449.fits.gz'


    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon   hgne  krypton  xenon  argon  ldls  quartz    quality  qual  stage  status  drpqual                    name
47     s    1111  60177  60177     flat  sp1     b1    3449    150.0  False  False    False  False  False  True   False  excellent     0      1       0        0  sdR-s-b1-00003449.fits
434    s    1111  60177  60177     flat  sp1     b1    3450    150.0  False  False    False  False  False  True   False  excellent     0      1       0        0  sdR-s-b1-00003450.fits


[INFO]: using header IMAGETYP = 'flat'
[INFO]: exposure time 150.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.71, 2.71, 2.68, 2.69] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 974.00 +/- 22.00 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 981.00 +/- 20.17 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 982.00 +/- 17.47 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 982.00 +/- 16.51 (ADU)
[INFO]: using header RDNOISE = [3.0, 3.0, 2.8, 2.8] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-b1.fits
[INFO]: building pixel mask
[INFO]: 106030 (0.64 %) pixels masked
[INFO]: replacing 106030 masked pixe

    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon   hgne  krypton  xenon  argon  ldls  quartz    quality  qual  stage  status  drpqual                    name
252    s    1111  60177  60177     flat  sp2     b2    3450    150.0  False  False    False  False  False  True   False  excellent     0      1       0        0  sdR-s-b2-00003450.fits
353    s    1111  60177  60177     flat  sp2     b2    3449    150.0  False  False    False  False  False  True   False  excellent     0      1       0        0  sdR-s-b2-00003449.fits


[INFO]: using header IMAGETYP = 'flat'
[INFO]: exposure time 150.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.45, 2.46, 2.46, 2.5] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 1003.00 +/- 8.10 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 1000.00 +/- 8.14 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 1007.00 +/- 7.60 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 1003.00 +/- 8.34 (ADU)
[INFO]: using header RDNOISE = [3.0, 3.0, 2.8, 2.8] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-b2.fits
[INFO]: building pixel mask
[INFO]: 170075 (1 %) pixels masked
[INFO]: replacing 170075 masked pixels w

...

[INFO]: using header IMAGETYP = 'flat'
[INFO]: exposure time 10.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.76, 2.65, 2.89, 2.78] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 980.00 +/- 10.22 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 979.00 +/- 11.27 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 990.00 +/- 536.64 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 990.00 +/- 12.60 (ADU)
[INFO]: using header RDNOISE = [3.6, 3.6, 3.4, 3.6] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-z1.fits
[INFO]: building pixel mask
[INFO]: 150150 (0.9 %) pixels masked
[INFO]: replacing 150150 masked pixel

    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
254    s    1111  60177  60177     flat  sp2     z2    3452     10.0  False  False    False  False  False  False    True  excellent     0      1       0        0  sdR-s-z2-00003452.fits
360    s    1111  60177  60177     flat  sp2     z2    3451     10.0  False  False    False  False  False  False    True  excellent     0      1       0        0  sdR-s-z2-00003451.fits


[INFO]: using header IMAGETYP = 'flat'
[INFO]: exposure time 10.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.6, 2.65, 2.57, 2.72] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 989.00 +/- 4.45 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 983.00 +/- 4.77 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 998.00 +/- 4.57 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 998.00 +/- 4.74 (ADU)
[INFO]: using header RDNOISE = [3.6, 3.6, 3.4, 3.6] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-z2.fits
[INFO]: building pixel mask
[INFO]: 155570 (0.93 %) pixels masked
[INFO]: replacing 155570 masked pixels wit

    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
45     s    1111  60177  60177     flat  sp3     z3    3451     10.0  False  False    False  False  False  False    True  excellent     0      1       0        0  sdR-s-z3-00003451.fits
390    s    1111  60177  60177     flat  sp3     z3    3452     10.0  False  False    False  False  False  False    True  excellent     0      1       0        0  sdR-s-z3-00003452.fits


[INFO]: using header IMAGETYP = 'flat'
[INFO]: exposure time 10.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.6, 2.65, 2.57, 2.72] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 1009.00 +/- 6.22 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 1013.00 +/- 6.55 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 1013.00 +/- 5.95 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 1010.00 +/- 6.83 (ADU)
[INFO]: using header RDNOISE = [3.6, 3.6, 3.4, 3.6] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-z3.fits
[INFO]: building pixel mask
[INFO]: 60465 (0.36 %) pixels masked
[INFO]: replacing 60465 masked pixels w

In [5]:
arc_analogs = frames_table.query("imagetyp=='arc'").groupby(["camera"])

marc_paths = []
for camera in arc_analogs.groups:
    arcs = arc_analogs.get_group(camera)
    arc = arcs.iloc[0].to_dict()
    
    lamp = MASTER_ARC_LAMPS[camera[0]]
    if camera[0] == "b":
        arcs = arcs.query("hgne & not neon & not argon & not xenon")
    else:
        arcs = arcs.query("neon & not hgne & not argon & not xenon")

    print(arcs.to_string())
    
    # define master paths for target frames
    mpixmask_path = os.path.join(masters_path, f"lvm-mpixmask-{camera}.fits")
    mbias_path = os.path.join(masters_path, f"lvm-mbias-{camera}.fits")
    mdark_path = os.path.join(masters_path, f"lvm-mdark-{camera}.fits")
    
    # define master frame path
    marc_path = path.full("lvm_master", drpver=drpver, tileid=arc["tileid"], mjd=masters_mjd, kind=f"marc_{lamp}", camera=arc["camera"])
    marc_paths.append(marc_path)
    os.makedirs(os.path.dirname(marc_path), exist_ok=True)
    
    # preprocess and detrend arcs
    darc_paths = []
    for arc in arcs.to_dict("records"):
        arc_path = path.full("lvm_raw", camspec=arc["camera"], **arc)
        parc_path = path.full("lvm_anc", drpver=drpver, kind="p", imagetype=arc["imagetyp"], **arc)
        darc_path = path.full("lvm_anc", drpver=drpver, kind="d", imagetype=arc["imagetyp"], **arc)
        darc_paths.append(darc_path)
        
        imageMethod.preproc_raw_frame(in_image=arc_path, out_image=parc_path, in_mask=mpixmask_path)
        imageMethod.detrend_frame(in_image=parc_path, out_image=darc_path, in_bias=mbias_path, in_dark=mdark_path, in_slitmap=SLITMAP)

    # create master arc (2D image)
    imageMethod.create_master_frame(in_images=darc_paths, out_image=marc_path, master_mjd=masters_mjd)

# cache new masters metadata into HDF5 files
new_masters = md.extract_metadata(frames_paths=marc_paths)
md.add_masters(new_masters)

[INFO]: starting preprocessing of raw image 'sdR-s-b1-00003463.fits.gz'


    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon  hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
59     s    1111  60177  60177      arc  sp1     b1    3463    150.0  False  True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00003463.fits
104    s    1111  60177  60177      arc  sp1     b1    3461     10.0  False  True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00003461.fits
290    s    1111  60177  60177      arc  sp1     b1    3464    150.0  False  True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00003464.fits
499    s    1111  60177  60177      arc  sp1     b1    3462     10.0  False  True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00003462.fits


[INFO]: using header IMAGETYP = 'arc'
[INFO]: exposure time 150.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.71, 2.71, 2.68, 2.69] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 974.00 +/- 1.13 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 981.00 +/- 1.15 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 982.00 +/- 1.30 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 982.00 +/- 1.29 (ADU)
[INFO]: using header RDNOISE = [3.0, 3.0, 2.8, 2.8] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-b1.fits
[INFO]: building pixel mask
[INFO]: 153403 (0.92 %) pixels masked
[INFO]: replacing 153403 masked pixels wi

    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon  hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
28     s    1111  60177  60177      arc  sp2     b2    3464    150.0  False  True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00003464.fits
63     s    1111  60177  60177      arc  sp2     b2    3461     10.0  False  True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00003461.fits
393    s    1111  60177  60177      arc  sp2     b2    3463    150.0  False  True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00003463.fits
416    s    1111  60177  60177      arc  sp2     b2    3462     10.0  False  True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00003462.fits


[INFO]: using header IMAGETYP = 'arc'
[INFO]: exposure time 150.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.45, 2.46, 2.46, 2.5] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 1003.00 +/- 1.13 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 1000.00 +/- 1.13 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 1007.00 +/- 1.25 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 1003.00 +/- 1.29 (ADU)
[INFO]: using header RDNOISE = [3.0, 3.0, 2.8, 2.8] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-b2.fits
[INFO]: building pixel mask
[INFO]: 208352 (1.2 %) pixels masked
[INFO]: replacing 208352 masked pixels 

...

[INFO]: using header IMAGETYP = 'arc'
[INFO]: exposure time 10.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.76, 2.65, 2.89, 2.78] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 980.00 +/- 1.09 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 979.00 +/- 1.11 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 989.00 +/- 1.20 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 990.00 +/- 1.20 (ADU)
[INFO]: using header RDNOISE = [3.6, 3.6, 3.4, 3.6] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-z1.fits
[INFO]: building pixel mask
[INFO]: 150150 (0.9 %) pixels masked
[INFO]: replacing 150150 masked pixels with

    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime  neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
100    s    1111  60177  60177      arc  sp2     z2    3458     10.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z2-00003458.fits
142    s    1111  60177  60177      arc  sp2     z2    3459    150.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z2-00003459.fits
208    s    1111  60177  60177      arc  sp2     z2    3457     10.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z2-00003457.fits
421    s    1111  60177  60177      arc  sp2     z2    3460    150.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z2-00003460.fits


[INFO]: starting preprocessing of raw image 'sdR-s-z2-00003458.fits.gz'
[INFO]: using header IMAGETYP = 'arc'
[INFO]: exposure time 10.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.6, 2.65, 2.57, 2.72] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 988.00 +/- 1.14 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 982.00 +/- 1.19 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 998.00 +/- 1.16 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 998.00 +/- 1.21 (ADU)
[INFO]: using header RDNOISE = [3.6, 3.6, 3.4, 3.6] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-z2.fits
[INFO]: building pixel mask
[INFO]: 1

    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime  neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
188    s    1111  60177  60177      arc  sp3     z3    3459    150.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z3-00003459.fits
265    s    1111  60177  60177      arc  sp3     z3    3458     10.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z3-00003458.fits
278    s    1111  60177  60177      arc  sp3     z3    3457     10.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z3-00003457.fits
303    s    1111  60177  60177      arc  sp3     z3    3460    150.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z3-00003460.fits


[INFO]: using header IMAGETYP = 'arc'
[INFO]: exposure time 150.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.6, 2.65, 2.57, 2.72] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 1009.00 +/- 1.20 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 1012.00 +/- 1.18 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 1012.00 +/- 1.30 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 1010.00 +/- 1.31 (ADU)
[INFO]: using header RDNOISE = [3.6, 3.6, 3.4, 3.6] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-z3.fits
[INFO]: building pixel mask
[INFO]: 101664 (0.61 %) pixels masked
[INFO]: replacing 101664 masked pixels